# Analyzing Amazon FineFood Reveiws

# Part 1: Preprocessing (preparing and cleaning data)

- However, we combined it with another dataset that contains all the US holidays 
- Our main dataset is 'Reviews.csv' file  retrieved from: https://www.kaggle.com/snap/amazon-fine-food-reviews 

In [6]:
#imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# read data into a DataFrame
dfFineFood = pd.read_csv("Reviews.csv")


#change time from unix to normal date format
dfFineFood['Time'] = pd.to_datetime(dfFineFood['Time'],unit='s')
# We Adapted Ameea's code for cleaning the data from:
# https://www.kaggle.com/ameeamin/d/snap/amazon-fine-food-reviews/using-k-means-clustering-to-predict-helpfulness/comments
dfFineFood['Summary'] = dfFineFood['Summary'].str.lower()
dfFineFood['Text'] = dfFineFood['Text'].str.lower()
dfFineFood['Score']= pd.to_numeric(dfFineFood['Score'])
# add new clomun which is a fraction of helpfulness value and the number of users 
dfFineFood['HelpfulnessFraction']=dfFineFood['HelpfulnessNumerator']/dfFineFood['HelpfulnessDenominator']
dfFineFood.HelpfulnessFraction[np.isnan(dfFineFood.HelpfulnessFraction)] = 0

#dfFineFood = dfFineFood.iloc[:, [6,7,8,9,10]]
dfFineFood.head()

/Users/majed/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,HelpfulnessFraction
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,good quality dog food,i have bought several of the vitality canned d...,1.0
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,not as advertised,product arrived labeled as jumbo salted peanut...,0.0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""delight"" says it all",this is a confection that has been around a fe...,1.0
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,cough medicine,if you are looking for the secret ingredient i...,1.0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,great taffy,great taffy at a great price. there was a wid...,0.0


# The other dataset is the holidays dataset , dates.csv

In [9]:
dfDates = pd.read_csv("dates.csv")
#make a copy of columns we need from holidays dataset 
dfDates = dfDates.iloc[:, [3,31,33]]
# rename the date column to "time" to align it with our main dataset then convert it to a right date format
dfDates.rename(columns={'FullDateUSA': 'Time'}, inplace=True)
dfDates['Time'] = pd.to_datetime(dfDates['Time'])
# merge them
mergeDataSet = pd.merge(dfDates,dfFineFood, on='Time', how='inner')
mergeDataSet.head(3)

/Users/majed/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (42,44,45,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Time,IsWeekday,IsHolidayUSA,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text,HelpfulnessFraction
0,1999-10-08,1,0,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,5,every book is educational,this witty little book makes my son laugh at l...,0.0
1,1999-10-25,1,0,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,5,this whole series is great way to spend time w...,i can remember seeing the show when it aired o...,1.0
2,1999-12-02,1,0,451856,B00004CXX9,AIUWLEQ1ADEG5,Elizabeth Medina,0,0,5,entertainingl funny!,beetlejuice is a well written movie ..... ever...,0.0


# Export the cleaned and merge data to upload it in github then run it in Spark 

In [10]:
mergeDataSet_copy= mergeDataSet
def lable(x):
    if x < 3:
        return 0
    return 1
mergeDataSet_copy['Score'] = mergeDataSet_copy['Score'].map(lable)
mergeDataSet_copy.to_csv('dfFineFood_copy.csv')
mergeDataSet_copy.head(2)

,Time,IsWeekday,IsHolidayUSA,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text,HelpfulnessFraction
0,1999-10-08,1,0,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,every book is educational,this witty little book makes my son laugh at l...,0.0
1,1999-10-25,1,0,150501,0006641040,AJ46FKXOVC7NR,Nicholas A Mesiano,2,2,1,this whole series is great way to spend time w...,i can remember seeing the show when it aired o...,1.0
